<img src="fine_tune_LLMs.jpg" width="70%">

HREF to Video

# -1 Research Background

<fieldset>
<legend>References</legend>
1. Hinton, G. E., Osindero, S., & Teh, Y. W. (2006). <a href="https://www.cs.toronto.edu/~hinton/absps/fastnc.pdf">A fast learning algorithm for deep belief nets.</a> <i>Neural computation</i>, 18(7), p. 1527-1554. <br>
2. Larochelle, H., Bengio, Y., Louradour, J., & Lamblin, P. (2009). <a href="https://www.jmlr.org/papers/volume10/larochelle09a/larochelle09a.pdf">Exploring strategies for training deep neural networks.</a> <i>Journal of machine learning research</i>, 10(1), p. 1-40.
</fieldset>

Snapshots from Larochelle et al. (2009) Exploring strategies for training deep neural networks.

<table style="border:1; border-color:red">
<th>
<td width=60% align='left'><img src="yoshio_bengio_2009_snapshot_1.jpg"></td>
<td align='left'><img src="yoshio_bengio_2009_snapshot_2.jpg" width=110%></td>
</th>    
</table>

# 0. Training Multi-Layer Perceptrons (MLPs): Pre-training and Fine-Tuning

<b>0.1 MLP Structure</b>

<img src="mlp.jpg">

<b>0.2 MLP Functionals Components</b>

<img src="mlp_functional_schema.jpg" width=60%>

<b>0.3 Restricted Boltzmann Machine (RMB)</b>

<img src="rbm.jpg">

<b>0.4 Stacked RMBs</b>

<img src="stacked_rbms.jpg" width=50%>

<b>0.5 Pre-train MLP with RBMs</b>

<img src="pre_train_MLP_with_RBMs.jpg" width=60%>

<b>0.6 Fine-Tuning Pre-Trained MLP</b>

<img src="fine_tuning_MLP.jpg" width=60%>

# 1. Transfer Learning 

<b>1.1 Transfer Learning: Basic Concept</b>

<img src="transfer_learning.jpg" width=60%>

<b>1.2 Convolutional Neural Networks: VGG16</b>

<img src="vgg16_architecture.jpg" width=60%>

<img src="vgg16_functional_schema.jpg" width=60%>

<b>1.2 Transfer Learning Quick Examples with VGG16</b>

<img src="transfer_learning_for_CNNs.jpg">

In [18]:
#load EuroSAT dataset
import numpy as np

euroSAT_load = np.load('euroSAT_dataset.npy',allow_pickle=True)

euroSAT_dataset = euroSAT_load[0].copy()

euroSAT_dataset.keys()

num_samples = len(euroSAT_dataset['labels'])

dataset_index = np.random.permutation(num_samples)

train_ratio = 0.7
valid_ratio = 0.15
test_ratio = 0.15

train_cut = int(num_samples*train_ratio)
valid_cut = int(num_samples*(train_ratio+valid_ratio))

train_ind = dataset_index[:train_cut]
valid_ind = dataset_index[train_cut:valid_cut]
test_ind = dataset_index[valid_cut:]

len(train_ind) + len(valid_ind) + len(test_ind)

train_ind = np.squeeze(np.array(train_ind))
valid_ind = np.squeeze(np.array(valid_ind))
test_ind = np.squeeze(np.array(test_ind))

train_img = euroSAT_dataset['images'][train_ind]
train_img = train_img/255

valid_img = euroSAT_dataset['images'][valid_ind]
valid_img = valid_img/255

test_img = euroSAT_dataset['images'][test_ind]
test_img = test_img/255

In [19]:
train_img.shape

(18900, 64, 64, 3)

In [20]:
eurosat_img_shape = train_img.shape[1:]
eurosat_img_shape 

(64, 64, 3)

In [21]:
import tensorflow as tf

In [22]:
from tensorflow import keras

In [23]:
train_labels = np.array(euroSAT_dataset['labels'])[train_ind]
train_labels = keras.utils.to_categorical(train_labels)

valid_labels = np.array(euroSAT_dataset['labels'])[valid_ind]
valid_labels = keras.utils.to_categorical(valid_labels)

In [24]:
train_labels[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

Step 1. Load trained model 

In [25]:
vgg16_model = keras.applications.vgg16.VGG16(weights='imagenet')

In [26]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [27]:
base_model = keras.applications.vgg16.VGG16(weights='imagenet',
                                            include_top=False,
                                            input_shape=eurosat_img_shape
                                           )

In [28]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [29]:
conv_base = keras.applications.vgg16.VGG16(weights='imagenet',
                                           include_top=False,
                                           input_shape=eurosat_img_shape)

transfer_layer = conv_base.get_layer('block5_pool')
conv_base.trainable = False

x = keras.layers.Flatten()(transfer_layer.output)
x = keras.layers.Dense(256)(x)
x = keras.layers.Dropout(0.5)(x)
pred = keras.layers.Dense(len(euroSAT_dataset['classes']), activation='softmax')(x)

tl_model = keras.Model(conv_base.input,pred)

In [30]:
tl_model.compile(loss="categorical_crossentropy",
                 optimizer="rmsprop",
                 metrics=["accuracy"])

In [31]:
#tl_model.build((180,180,3))
tl_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0   

In [32]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [33]:
tf.__version__

'2.13.0'

In [34]:
history = tl_model.fit(train_img,
                       train_labels,
                       batch_size=16,
                       epochs=20,
#                       steps_per_epoch=100,
#                       #verbose="auto",
#                       #callbacks=None,
                       validation_data=(valid_img, valid_labels),
#                       validation_steps=100
                  )

Epoch 1/20
1182/1182 [==============================] - 213s 180ms/step - loss: 0.9408 - accuracy: 0.7129 - val_loss: 0.5246 - val_accuracy: 0.8227
Epoch 2/20
1182/1182 [==============================] - 224s 190ms/step - loss: 0.6537 - accuracy: 0.7999 - val_loss: 0.6944 - val_accuracy: 0.7773
Epoch 3/20
1182/1182 [==============================] - 218s 185ms/step - loss: 0.5907 - accuracy: 0.8170 - val_loss: 0.5171 - val_accuracy: 0.8353
Epoch 4/20
1182/1182 [==============================] - 220s 186ms/step - loss: 0.5407 - accuracy: 0.8321 - val_loss: 0.5308 - val_accuracy: 0.8326
Epoch 5/20
1182/1182 [==============================] - 221s 187ms/step - loss: 0.5182 - accuracy: 0.8384 - val_loss: 0.6146 - val_accuracy: 0.8151
Epoch 6/20
1182/1182 [==============================] - 218s 185ms/step - loss: 0.4999 - accuracy: 0.8500 - val_loss: 0.4939 - val_accuracy: 0.8479
Epoch 7/20
1182/1182 [==============================] - 216s 183ms/step - loss: 0.4821 - accuracy: 0.8515 - val_

In [35]:
history1 = tl_model.fit(train_img,
                       train_labels,
                       batch_size=16,
                       epochs=20,
#                       steps_per_epoch=100,
#                       #verbose="auto",
#                       #callbacks=None,
                       validation_data=(valid_img, valid_labels),
#                       validation_steps=100
                  )

Epoch 1/20
1182/1182 [==============================] - 232s 196ms/step - loss: 0.3940 - accuracy: 0.8845 - val_loss: 0.4898 - val_accuracy: 0.8627
Epoch 2/20
1182/1182 [==============================] - 219s 185ms/step - loss: 0.3921 - accuracy: 0.8820 - val_loss: 0.4926 - val_accuracy: 0.8640
Epoch 3/20
1182/1182 [==============================] - 216s 183ms/step - loss: 0.3875 - accuracy: 0.8844 - val_loss: 0.4657 - val_accuracy: 0.8723
Epoch 4/20
1182/1182 [==============================] - 217s 184ms/step - loss: 0.3873 - accuracy: 0.8812 - val_loss: 0.4904 - val_accuracy: 0.8625
Epoch 5/20
1182/1182 [==============================] - 216s 183ms/step - loss: 0.3817 - accuracy: 0.8830 - val_loss: 0.5841 - val_accuracy: 0.8506
Epoch 6/20
1182/1182 [==============================] - 216s 183ms/step - loss: 0.3824 - accuracy: 0.8861 - val_loss: 0.4686 - val_accuracy: 0.8716
Epoch 7/20
1182/1182 [==============================] - 217s 184ms/step - loss: 0.3739 - accuracy: 0.8859 - val_

In [ ]:
import matplotlib.pyplot

## 0. TRANSFORMER vs. BERT Architecture

<img src="transformers.jpg" width="70%">

In [ ]:
mini_imgnet = load_dataset("zh-plus/tiny-imagenet")

In [ ]:
type(mini_imgnet['valid']['label'][0])

In [ ]:
type(mini_imgnet['valid']['image'])

In [ ]:
valid_array = np.array([])
valid_imgs = mini_imgnet['valid']['image'].copy()
for i, img in enumerate(valid_imgs):
    new_image = img.resize((200, 200))
    valid_array = np.stack([valid_array,np.array(new_image)])

In [ ]:
valid_array = np.array(valid_imgs)

In [ ]:
train_imgs = mini_imgnet['train']['image'].copy()
for i, img in enumerate(train_imgs):
    train_imgs[i] = np.array(img)

In [ ]:
train_labels = keras.utils.to_categorical(mini_imgnet['train']['label'],
                                         num_classes=200)
valid_labels = keras.utils.to_categorical(mini_imgnet['valid']['label'],
                                         num_classes=200)

### Fine-tune BERT Base model for Support Ticket Dataset Classification

In [ ]:
from datasets import load_dataset
from datasets import DatasetDict, Dataset

import matplotlib.pyplot as plt

from transformers import (logging,
    AutoTokenizer, DataCollatorWithPadding, 
    AutoModelForSequenceClassification, TrainingArguments, Trainer
    )
import evaluate
import numpy as np
import pandas as pd

# ignore warnings from transformers lib
logging.set_verbosity_error() 

### 1. Load data from Support Tickets Dataset

<b>1.1 Load Support Tickets Dataset fom HuggingFace Hub</b>

In [ ]:
support_tickets = load_dataset("phi-ai-info/support_tickets",name='alpha')

In [ ]:
support_tickets

In [ ]:
def dataset_to_df(dataset_obj, 
                  label=None,
                  reference='class',
                  new_key='label'
                 ):
    cols = list(dataset_obj[0].keys())

    cols_values = {item:[] for item in cols}
    for json_rec in dataset_obj:
        for item in cols:
            cols_values[item].append(json_rec[item])
    
    if label is not None:
        cols += [new_key]
        cols_values[new_key] = []
        for json_rec in dataset_obj:
            cols_values[new_key].append(label[json_rec[reference]])
    
    return pd.DataFrame({item:cols_values[item] for item in cols})

In [ ]:
label_class_dict = {'revoke access': 'access',
 'grant access': 'access',
 'access profile': 'access',
 'add user': 'user',
 'delete user': 'user',
 'create user': 'user',
 'modify user': 'user',
 'user role': 'user',
 'disk space': 'storage',
 'hard disk': 'storage',
 'disk full': 'storage',
 'ssd disk': 'storage',
 'disk error': 'storage',
 'shared disk': 'storage',
 'nas disk': 'storage',
 'printer functioning': 'printer',
 'printer driver': 'printer',
 'printer toner': 'printer',
 'printer paper': 'printer',
 'wifi functioning': 'network',
 'network functioning': 'network',
 'email server': 'servers',
 'web server': 'servers'}

In [ ]:
label_name_dict = {item: i for i,item in enumerate(list(set(label_class_dict.values())))}
label_name_dict

In [ ]:
label_dict = {'grant access': 0,
 'revoke access': 0,
 'access profile': 0,
 'disk space': 1,
 'disk full': 1,
 'disk error': 1,
 'add user': 2,
 'delete user': 2,
 'create user': 2,
 'modify user': 2}

In [ ]:
df_set = dataset_to_df(support_tickets['train'], 
                       label=label_name_dict)

In [ ]:
df_set.head()


df_set

In [ ]:
random_index = np.random.permutation(df_set.index)
random_index

In [ ]:
subsets = ['train','valid','test']
train_ratio = 0.6
lng = len(random_index)
train_start = 0
train_end = int(lng*train_ratio)

val_ratio = 0.2
val_start = train_end+1
val_end = int(lng*(train_ratio+val_ratio))

test_start = val_end + 1 
test_end = lng

split_ind = np.array([[train_start,train_end],
                      [val_start,val_end],
                      [test_start,test_end]])

ind2subset = {key:random_index[split_ind[i][0]:split_ind[i][1]] for i,key in enumerate(subsets)}
ind2subset

In [ ]:
df_set.iloc[ind2subset['valid']]['label'].unique()

<b>1.3 Create a Dataset with Train, Validation and Test splits</b>

In [ ]:
ds_splits = DatasetDict({
                        'train': Dataset.from_pandas(
                                                    df_set.iloc[ind2subset['train']]
                                                    ),
                        'valid': Dataset.from_pandas(
                                                    df_set.iloc[ind2subset['valid']]
                                                    ),
                        'test': Dataset.from_pandas(
                                                    df_set.iloc[ind2subset['test']]
                                                    )
                        })
ds_splits

In [ ]:
subsets = ['train','valid','test']
split_dict = { key: Dataset.from_pandas(df_set.iloc[ind2subset[key]]) for key in subsets}
support_tickets_splits = DatasetDict(split_dict)
support_tickets_splits

### 2. Train BERT base Model

<b>2.1 Load model from HuggingFace Hub</b>

In [ ]:
label_name_dict

In [ ]:
model_path = "google-bert/bert-base-uncased"
#model_path = 'distilbert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_path)

label2id = label_name_dict.copy()
id2label = {label2id[key]:key for key in label2id}
#id2label = {0: "access", 1: "disk", 2: "user"}
#label2id = {id2label[key]:key for key in id2label}

model = AutoModelForSequenceClassification.from_pretrained(model_path, 
                                                           num_labels=len(list(id2label.keys())), 
                                                           id2label=id2label, 
                                                           label2id=label2id
                                                          )

In [ ]:
label2id

In [ ]:
id2label

<b>2.2 Viz Model Stru</b>

In [ ]:
# print layers
for name, param in model.named_parameters():
   print(name, param.requires_grad)

<b>2.3 Freeze base model</b>

In [ ]:
# freeze base model parameters and unfreeze base model pooling layers
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

In [ ]:
# print layers
for name, param in model.named_parameters():
   print(name, param.requires_grad)

#### 3. Training Helpers

<b>3.1 Preprocess Text Data</b>

In [ ]:
# define text preprocessing
def preprocess_function(examples):
    return tokenizer(examples["description"], truncation='only_first')

In [ ]:
# tokenize all datasetse
tokenized_data = support_tickets_splits.map(preprocess_function, batched=True)

In [ ]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

<b>3.2 Evaluation metrics</b>

In [ ]:
# load metrics
accuracy = evaluate.load("accuracy")
#auc_score = evaluate.load("roc_auc")

def compute_softmax(preds):
    max_val = preds.max()
    pred_exp = np.exp(preds-max_val)
    probs = pred_exp/pred_exp.sum(-1, keepdims=True)
    return probs 


def compute_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred
    
    # apply softmax to get probabilities
    probabilities = compute_softmax(predictions)
    # use probabilities of the positive class for ROC AUC
    positive_class_probs = probabilities[:, 1]
    # compute auc
    #auc = float(round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'],3))
    
    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = float(round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3))
    
    return {"Accuracy": acc}#, "AUC": auc}

#### 4. Train model

<b>4.1 Set Training Arguments</b>

In [ ]:
# hyperparameters
lr = 1e-2
batch_size = 8
num_epochs = 100

training_args = TrainingArguments(
    output_dir="bert_ticket_classifier",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    use_cpu=False
)

In [ ]:
training_args

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

<b>4.2 Define Trainer Module</b>

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
import torch

In [ ]:
#trainer.__dict__['args'].device = torch.device('cpu')
trainer.args.device

<b>4.3 Fine-tune BERT Model</b>

In [ ]:
trainer.train()

<b>4.4 Extract Training History</b>

In [ ]:
trainer.state.log_history[18]

In [ ]:
trainer.state.log_history[19]

In [ ]:
trainer.state.log_history[20]

In [ ]:
len(trainer.state.log_history[:-1]),len(trainer.state.log_history)

In [ ]:
learning_history = trainer.state.log_history[:-1]

In [ ]:
hist_dict = {
'train_loss': [],
'valid_loss': [],
'valid_acc': []
}

for item in learning_history:
    if 'loss' in item:
        hist_dict['train_loss'].append(item['loss'])
        continue
        
    if 'eval_loss' in item:
        hist_dict['valid_loss'].append(item['eval_loss'])
        hist_dict['valid_acc'].append(item['eval_Accuracy'])

In [ ]:
for key in  hist_dict:
    print(key,hist_dict[key])

In [ ]:
plt.subplot(1,2,1)
for key in  list(hist_dict.keys())[:-1]:
    plt.plot(hist_dict[key])
plt.legend(list(hist_dict.keys())[:-1])
plt.title('Train vs. Validation Loss')
plt.xlabel('Epochs')

plt.subplot(1,2,2)
for key in  [list(hist_dict.keys())[-1]]:
    plt.plot(hist_dict[key])
plt.legend([list(hist_dict.keys())[-1]])
plt.title('Validation Acc')
plt.xlabel('Epochs')

<b> 4.5 Apply Model to Validation Dataset</b>

In [ ]:
# apply model to validation dataset
split_key = 'test'

predictions = trainer.predict(tokenized_data[split_key])

# Extract the logits and labels from the predictions object
logits = predictions.predictions
labels = predictions.label_ids

# Use your compute_metrics function
#metrics = compute_metrics((logits, labels))
#print(metrics)
true_count = 0
for pred_l,actu_l in zip(labels,support_tickets_splits[split_key]['label']): 
    if pred_l == actu_l:
        true_count += 1
print(f'Accuracy {true_count/len(labels)}')

In [ ]:
labels

In [ ]:
support_tickets_splits['test']['label']

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns

font_size = 12
label_class = list(label2id.keys())
cl_rep = classification_report(support_tickets_splits['test']['label'],
                               labels,
                               target_names=label_class,
                               output_dict=True
                            )

cm = confusion_matrix(support_tickets_splits['test']['label'],
                      labels)

plt.figure(figsize = (8, 5))
sns.heatmap(cm,
            xticklabels=label_class,
            yticklabels=label_class,
            annot = True,  fmt = '.0f',
            annot_kws={'size': font_size})
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
df = pd.DataFrame(cl_rep)
df

In [ ]:
# # push model to hub
# trainer.push_to_hub()

## LORA

In [ ]:
#model_path = "google-bert/bert-base-uncased"
model_path_dict = {}
model_path_dict['lora'] = 'distilbert-base-uncased'

label2id = label_name_dict.copy()
id2label = {label2id[key]:key for key in label2id}
#id2label = {0: "access", 1: "disk", 2: "user"}
#label2id = {id2label[key]:key for key in id2label}

model_dict = {}

model_dict['lora'] = AutoModelForSequenceClassification.from_pretrained(model_path_dict['lora'], 
                                                                        num_labels=len(list(id2label.keys())), 
                                                                        id2label=id2label, 
                                                                        label2id=label2id
                                                                       )

In [ ]:
# create tokenizer
tokenizer_dict = {}
tokenizer_dict['lora'] = AutoTokenizer.from_pretrained(model_path_dict['lora'], 
                                                       add_prefix_space=True)

# add pad token if none exists
if tokenizer_dict['lora'].pad_token is None:
    tokenizer_dict['lora'].add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer_dict['lora']))

In [ ]:
tokenizer_dict['lora']

In [ ]:
# create tokenize function
def tokenize_function_lora(examples):
    # extract text
    text = examples["description"]

    #tokenize and truncate text
    tokenizer_dict['lora'].truncation_side = "left"
    tokenized_inputs = tokenizer_dict['lora'](text,
                                              return_tensors="np",
                                              truncation=True,
                                              max_length=512
                                             )

    return tokenized_inputs

In [ ]:
# tokenize training and validation datasets

tokenized_dataset_dict = {}
tokenized_dataset_dict['lora'] = ds_splits.map(tokenize_function_lora, 
                                               batched=True)
tokenized_dataset_dict['lora']

In [ ]:
# create data collator
data_collator_dict = {}

data_collator_dict['lora'] = DataCollatorWithPadding(tokenizer=tokenizer_dict['lora'])

In [ ]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

# define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

In [ ]:
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate

In [ ]:
for name, param in model_lora.named_parameters():
   print(name, param.requires_grad)

In [ ]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['q_lin','k_lin','v_lin'])

In [ ]:
model_dict['lora'] = get_peft_model(model_dict['lora'], 
                                    peft_config)
model_dict['lora'].print_trainable_parameters()

In [ ]:
# hyperparameters
lr = 1e-3
batch_size = 16
num_epochs = 10

In [ ]:
train_arg_dict = {}
# define training arguments
train_arg_dict['lora'] = TrainingArguments(
                                            output_dir= model_path_dict['lora'] + "-lora-text-classification",
                                            learning_rate=lr,
                                            per_device_train_batch_size=batch_size,
                                            per_device_eval_batch_size=batch_size,
                                            num_train_epochs=num_epochs,
                                            weight_decay=0.01,
                                            eval_strategy="epoch",
                                            save_strategy="epoch",
                                            load_best_model_at_end=True,
                                        )

In [ ]:
trainer_dict = {}
trainer_dict['lora'] = Trainer(model=model_dict['lora'],
                               args=train_arg_dict['lora'],
                               train_dataset=tokenized_dataset_dict['lora']["train"],
                               eval_dataset=tokenized_dataset_dict['lora']["valid"],
                               tokenizer=tokenizer_dict['lora'],
                               data_collator=data_collator_dict['lora'],
                               compute_metrics=compute_metrics)

In [ ]:
trainer_dict['lora'].train()

In [ ]:
split_key = "test"

# apply model to validation dataset
predictions = trainer.predict(tokenized_data[split_key])

# Extract the logits and labels from the predictions object
logits = predictions.predictions
labels = predictions.label_ids

# Use your compute_metrics function
#metrics = compute_metrics((logits, labels))
#print(metrics)
true_count = 0
for pred_l,actu_l in zip(labels,support_tickets_splits[split_key]['label']): 
    if pred_l == actu_l:
        true_count += 1
print(f'Accuracy {true_count/len(labels)}')

In [ ]:
font_size = 12
label_class = list(label2id.keys())
cl_rep = classification_report(support_tickets_splits['test']['label'],
                               labels,
                               target_names=label_class,
                               output_dict=True
                            )

cm = confusion_matrix(support_tickets_splits['test']['label'],
                      labels)

plt.figure(figsize = (8, 5))
sns.heatmap(cm,
            xticklabels=label_class,
            yticklabels=label_class,
            annot = True,  fmt = '.0f',
            annot_kws={'size': font_size})
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()